In [1]:
import pandas as pd
import numpy as np

In [2]:
#impressions_df_2022_q3 = pd.read_csv("./impressions_2022_q3.csv")
#impressions_df_2022_q4 = pd.read_csv("./impressions_2022_q4.csv")
#impressions_df_2023_q1 = pd.read_csv("./impressions_2023_q1.csv")

#impressions = pd.concat([impressions_df_2022_q3, impressions_df_2022_q4, impressions_df_2023_q1])
impressions = pd.read_csv("./impressions_2022_q3_q4_brands.csv")
impressions = impressions[impressions["gaze_valid"] == True]

In [3]:
impressions["true_is_iab_inview"] = impressions.apply(
    lambda x: 
    True if (
        (x["ad_technical_format"] == "out-stream") & (x["exist_viewable_1_s_threshold_50"] == True) & (x["exist_viewable_2_s_threshold_50"] == False)
    ) | (x["is_iab_inview"] == True) 
    else False, 
    axis=1
)

In [4]:
def get_size(x):
    return f"{x['ad_width_chosen']}x{x['ad_height_chosen']}"

impressions["size"] = impressions.apply(lambda x: get_size(x), axis=1)

In [5]:
from datetime import datetime
def get_day(x):
    date = datetime.strptime(x["first_event_at_iso"].split("T")[0], "%Y-%m-%d")
    return date.strftime("%A")

def get_week(x):
    date = datetime.strptime(x["first_event_at_iso"].split("T")[0], "%Y-%m-%d")
    return date.strftime("%V")

impressions["day"] = impressions.apply(lambda x: get_day(x), axis=1)
impressions["week"] = impressions.apply(lambda x: get_week(x), axis=1)

In [6]:
impressions["chosen_brand"].value_counts()

klarna        85112
amazon        37961
samsung       32506
elgiganten    26650
netonnet      16667
              ...  
oppigards         1
stiga             1
garnier           1
bonaqua           1
bank24            1
Name: chosen_brand, Length: 394, dtype: int64

In [20]:
grouped_impressions = impressions.groupby(["part_year", "part_month", "hostname", "size", "chosen_brand"]).apply(
    lambda row: pd.Series(
        {
            "num_impressions": row["id"].count(),
            "num_fixations": row[row["is_fixated"] == True]["id"].count(),
            "total_fixation_duration": row["total_fixation_duration"].sum(),
            "num_inviews": row.loc[row["true_is_iab_inview"], "id"].count(),
        }
    )
)
grouped_impressions = grouped_impressions.reset_index()
grouped_impressions = grouped_impressions[grouped_impressions["num_impressions"] >= 100]
grouped_impressions["imp/fix_ratio"] = round(grouped_impressions["num_fixations"] / grouped_impressions["num_impressions"] * 100, 2)

grouped_impressions

,part_year,part_month,hostname,size,chosen_brand,num_impressions,num_fixations,total_fixation_duration,num_inviews,imp/fix_ratio
56,2022,7,aftonbladet.se,1180x240,amazon,241,166,1413431,107,68.88
80,2022,7,aftonbladet.se,160x600,comviq,100,3,2198,63,3.00
83,2022,7,aftonbladet.se,160x600,elgiganten,114,9,4173,88,7.89
101,2022,7,aftonbladet.se,160x600,lendo,371,24,14928,256,6.47
102,2022,7,aftonbladet.se,160x600,lendototal,371,24,14928,256,6.47
...,...,...,...,...,...,...,...,...,...,...
50450,2022,12,youtube.com,468x120,disneyplus,150,10,5687,58,6.67
50622,2022,12,youtube.com,980x360,disneyplus,107,40,120238,22,37.38
50768,2022,12,zkillboard.com,1180x240,videoslots,104,9,1912,25,8.65
50793,2022,12,zkillboard.com,160x600,snabbare,208,34,109747,206,16.35


In [22]:
final_df = grouped_impressions[["part_year", "part_month", "hostname", "size", "chosen_brand", "num_impressions", "num_fixations", "total_fixation_duration", "num_inviews", "imp/fix_ratio"]]
final_df = final_df.rename(
    columns={
        "part_year": "Year",
        "part_month": "Month",
        "hostname": "Hostname",
        "size": "Size",
        "chosen_brand": "Brand",
        "num_impressions": "Impressions", 
        "num_fixations": "Fixations",
        "num_inviews": "Inviews",
        "imp/fix_ratio": "Fixation ratio %"
    }
)
final_df.to_excel("2022_q3_q4_impressions.xlsx", index=False)

In [8]:
facebook_imps = grouped_impressions[grouped_impressions["hostname"] == "facebook.com"]
facebook_imps["imp/fix_ratio"] = round(facebook_imps["num_fixations"] / facebook_imps["num_impressions"] * 100, 2)
facebook_imps.sort_values(by=["total_fixation_duration"], ascending=False)

/tmp/ipykernel_17058/2500603168.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  facebook_imps["imp/fix_ratio"] = round(facebook_imps["num_fixations"] / facebook_imps["num_impressions"] * 100, 2)


,part_year,part_month,hostname,size,chosen_brand,num_impressions,num_fixations,total_fixation_duration,num_inviews,imp/fix_ratio
11337,2022,8,facebook.com,500x555,actsvenskakyrkan,19,15,6054168,17,78.95
11402,2022,8,facebook.com,500x555,mathem,8,6,4980611,4,75.00
11423,2022,8,facebook.com,500x555,pepsi,3,3,3734631,3,100.00
11446,2022,8,facebook.com,500x555,sverigedemokraterna,179,89,3568937,65,49.72
11394,2022,8,facebook.com,500x555,kristdemokraterna,112,68,3285575,54,60.71
...,...,...,...,...,...,...,...,...,...,...
11262,2022,8,facebook.com,285x230,friskissvettis,1,0,0,0,0.00
2472,2022,7,facebook.com,285x230,porsche,4,0,0,2,0.00
21750,2022,9,facebook.com,500x555,valio,1,0,0,1,0.00
11265,2022,8,facebook.com,285x230,honda,1,0,0,1,0.00


In [9]:
# Topical ads generate the most fixation ratios, not the largest
# facebook_imps[facebook_imps["week"] == "49"].sort_values(by=["imp/fix_ratio"], ascending=False)

In [10]:
prisjakt_imps = grouped_impressions[grouped_impressions["hostname"] == "prisjakt.nu"]
#prisjakt_imps
prisjakt_imps["imp/fix_ratio"] = round(prisjakt_imps["num_fixations"] / prisjakt_imps["num_impressions"] * 100, 2)
prisjakt_imps.sort_values(by=["imp/fix_ratio"], ascending=False)


/tmp/ipykernel_17058/1510168053.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prisjakt_imps["imp/fix_ratio"] = round(prisjakt_imps["num_fixations"] / prisjakt_imps["num_impressions"] * 100, 2)


,part_year,part_month,hostname,size,chosen_brand,num_impressions,num_fixations,total_fixation_duration,num_inviews,imp/fix_ratio
47960,2022,12,prisjakt.nu,980x360,webhallen,1,1,7501,0,100.0
37216,2022,11,prisjakt.nu,980x120,bmw,1,1,700,1,100.0
15501,2022,8,prisjakt.nu,250x480,oneblade,1,1,3862,1,100.0
15513,2022,8,prisjakt.nu,980x120,husqvarna,1,1,123,1,100.0
26172,2022,9,prisjakt.nu,250x480,audi,1,1,134,1,100.0
...,...,...,...,...,...,...,...,...,...,...
26189,2022,9,prisjakt.nu,728x90,skoda,1,0,0,0,0.0
26188,2022,9,prisjakt.nu,728x90,nissan,1,0,0,0,0.0
26187,2022,9,prisjakt.nu,728x90,honda,1,0,0,0,0.0
26186,2022,9,prisjakt.nu,728x90,ford,1,0,0,0,0.0


In [11]:
telia_imps = grouped_impressions[grouped_impressions["chosen_brand"] == "telia"]
telia_imps["imp/fix_ratio"] = round(telia_imps["num_fixations"] / telia_imps["num_impressions"] * 100, 2)

tele2_imps = grouped_impressions[grouped_impressions["chosen_brand"] == "tele2"]
tele2_imps["imp/fix_ratio"] = round(tele2_imps["num_fixations"] / tele2_imps["num_impressions"] * 100, 2)

volvo_imps = grouped_impressions[grouped_impressions["chosen_brand"] == "volvo"]
volvo_imps["imp/fix_ratio"] = round(volvo_imps["num_fixations"] / volvo_imps["num_impressions"] * 100, 2)

mercedes_imps = grouped_impressions[grouped_impressions["chosen_brand"] == "mercedes"]
mercedes_imps["imp/fix_ratio"] = round(mercedes_imps["num_fixations"] / mercedes_imps["num_impressions"] * 100, 2)

ford_imps = grouped_impressions[grouped_impressions["chosen_brand"] == "ford"]
ford_imps["imp/fix_ratio"] = round(ford_imps["num_fixations"] / ford_imps["num_impressions"] * 100, 2)

elgiganten_imps = grouped_impressions[grouped_impressions["chosen_brand"] == "elgiganten"]
elgiganten_imps["imp/fix_ratio"] = round(elgiganten_imps["num_fixations"] / elgiganten_imps["num_impressions"] * 100, 2)

samsung_imps = grouped_impressions[grouped_impressions["chosen_brand"] == "samsung"]
samsung_imps["imp/fix_ratio"] = round(samsung_imps["num_fixations"] / samsung_imps["num_impressions"] * 100, 2)

/tmp/ipykernel_17058/4005354330.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  telia_imps["imp/fix_ratio"] = round(telia_imps["num_fixations"] / telia_imps["num_impressions"] * 100, 2)
/tmp/ipykernel_17058/4005354330.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tele2_imps["imp/fix_ratio"] = round(tele2_imps["num_fixations"] / tele2_imps["num_impressions"] * 100, 2)
/tmp/ipykernel_17058/4005354330.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Tr

In [12]:
# Wider ads generate highest fixation ratio on news sites
telia_imps[telia_imps["num_impressions"] > 100].sort_values(by=["imp/fix_ratio"], ascending=False)

,part_year,part_month,hostname,size,chosen_brand,num_impressions,num_fixations,total_fixation_duration,num_inviews,imp/fix_ratio
31424,2022,11,di.se,980x240,telia,134,105,193882,132,78.36
18675,2022,9,aftonbladet.se,468x240,telia,131,76,175262,81,58.02
451,2022,7,aftonbladet.se,468x240,telia,147,84,147179,110,57.14
21379,2022,9,expressen.se,468x480,telia,119,64,73581,85,53.78
29668,2022,11,aftonbladet.se,468x240,telia,176,85,263920,116,48.30
40870,2022,12,aftonbladet.se,468x240,telia,189,84,178911,115,44.44
2279,2022,7,expressen.se,468x480,telia,103,42,70738,59,40.78
7194,2022,7,telia.se,180x150,telia,133,0,0,0,0.00
18433,2022,9,aftonbladet.se,180x150,telia,104,0,0,0,0.00
27391,2022,9,telia.se,180x150,telia,135,0,0,0,0.00


In [13]:
# Wider ads generate highest fixation ratio on news sites
tele2_imps[tele2_imps["num_impressions"] > 100].sort_values(by=["imp/fix_ratio"], ascending=False)

,part_year,part_month,hostname,size,chosen_brand,num_impressions,num_fixations,total_fixation_duration,num_inviews,imp/fix_ratio
32404,2022,11,expressen.se,980x360,tele2,112,55,77880,59,49.11
40868,2022,12,aftonbladet.se,468x240,tele2,177,80,57264,91,45.20
18673,2022,9,aftonbladet.se,468x240,tele2,107,38,36639,44,35.51
29798,2022,11,aftonbladet.se,980x240,tele2,156,55,38935,106,35.26
570,2022,7,aftonbladet.se,980x240,tele2,233,77,130793,147,33.05
6246,2022,7,prisjakt.nu,980x120,tele2,102,28,20727,83,27.45
1699,2022,7,dn.se,160x600,tele2,104,28,194191,86,26.92
26203,2022,9,prisjakt.nu,980x120,tele2,124,33,24980,90,26.61
18798,2022,9,aftonbladet.se,980x240,tele2,129,34,53514,74,26.36
37244,2022,11,prisjakt.nu,980x120,tele2,111,26,29980,82,23.42


In [14]:
volvo_imps[volvo_imps["num_impressions"] > 100].sort_values(by=["imp/fix_ratio"], ascending=False)

,part_year,part_month,hostname,size,chosen_brand,num_impressions,num_fixations,total_fixation_duration,num_inviews,imp/fix_ratio
18681,2022,9,aftonbladet.se,468x240,volvo,116,49,71257,71,42.24
28710,2022,9,youtube.com,300x250,volvo,579,78,38926,299,13.47
17803,2022,8,youtube.com,300x250,volvo,227,21,8737,106,9.25
9472,2022,8,blocket.se,180x150,volvo,121,0,0,0,0.00
16868,2022,8,tradera.com,980x240,volvo,108,0,0,32,0.00
17761,2022,8,youtube.com,180x150,volvo,625,0,0,0,0.00
19513,2022,9,blocket.se,180x150,volvo,238,0,0,0,0.00
28679,2022,9,youtube.com,180x150,volvo,1452,0,0,0,0.00
30380,2022,11,blocket.se,180x150,volvo,205,0,0,0,0.00
41605,2022,12,blocket.se,180x150,volvo,318,0,0,0,0.00


In [15]:
mercedes_imps[mercedes_imps["num_impressions"] > 100].sort_values(by=["imp/fix_ratio"], ascending=False)

,part_year,part_month,hostname,size,chosen_brand,num_impressions,num_fixations,total_fixation_duration,num_inviews,imp/fix_ratio
49173,2022,12,thefashionball.com,300x250,mercedes,122,19,7206,81,15.57
30347,2022,11,blocket.se,180x150,mercedes,180,0,0,0,0.00
41574,2022,12,blocket.se,180x150,mercedes,338,0,0,0,0.00


In [16]:
ford_imps[ford_imps["num_impressions"] > 100].sort_values(by=["imp/fix_ratio"], ascending=False)

,part_year,part_month,hostname,size,chosen_brand,num_impressions,num_fixations,total_fixation_duration,num_inviews,imp/fix_ratio
40824,2022,12,aftonbladet.se,468x240,ford,337,165,223002,223,48.96
8771,2022,8,aftonbladet.se,468x240,ford,149,52,34265,84,34.90
29610,2022,11,aftonbladet.se,468x240,ford,584,200,180579,349,34.25
18620,2022,9,aftonbladet.se,468x240,ford,413,126,239567,256,30.51
33341,2022,11,google.com,468x240,ford,129,32,11010,78,24.81
36696,2022,11,omni.se,980x240,ford,170,23,6030,119,13.53
20546,2022,9,di.se,468x480,ford,112,7,11764,6,6.25
30330,2022,11,blocket.se,180x150,ford,189,0,0,0,0.00
42149,2022,12,comick.app,980x240,ford,110,0,220,10,0.00


In [17]:
elgiganten_imps[elgiganten_imps["num_impressions"] > 100].sort_values(by=["imp/fix_ratio"], ascending=False)

,part_year,part_month,hostname,size,chosen_brand,num_impressions,num_fixations,total_fixation_duration,num_inviews,imp/fix_ratio
37082,2022,11,prisjakt.nu,1180x240,elgiganten,192,151,1034690,166,78.65
12376,2022,8,google.com,468x480,elgiganten,126,75,545862,103,59.52
22574,2022,9,google.com,468x240,elgiganten,159,77,304743,121,48.43
40819,2022,12,aftonbladet.se,468x240,elgiganten,242,116,119518,118,47.93
33479,2022,11,google.com,468x480,elgiganten,1172,549,2436715,781,46.84
22729,2022,9,google.com,468x480,elgiganten,287,134,829811,204,46.69
8768,2022,8,aftonbladet.se,468x240,elgiganten,190,87,104241,79,45.79
44566,2022,12,google.com,468x480,elgiganten,585,244,966793,391,41.71
33333,2022,11,google.com,468x240,elgiganten,427,178,725958,270,41.69
29604,2022,11,aftonbladet.se,468x240,elgiganten,134,55,95635,76,41.04


In [18]:
samsung_imps[samsung_imps["num_impressions"] > 100].sort_values(by=["imp/fix_ratio"], ascending=False)

,part_year,part_month,hostname,size,chosen_brand,num_impressions,num_fixations,total_fixation_duration,num_inviews,imp/fix_ratio
22650,2022,9,google.com,468x240,samsung,166,114,118376,117,68.67
40298,2022,11,youtube.com,980x360,samsung,103,56,178662,24,54.37
22801,2022,9,google.com,468x480,samsung,372,190,2698556,272,51.08
33861,2022,11,google.com,980x360,samsung,132,66,121884,90,50.00
438,2022,7,aftonbladet.se,468x240,samsung,175,87,126367,108,49.71
...,...,...,...,...,...,...,...,...,...,...
19495,2022,9,blocket.se,180x150,samsung,114,0,0,0,0.00
18416,2022,9,aftonbladet.se,180x150,samsung,463,0,0,0,0.00
17742,2022,8,youtube.com,180x150,samsung,116,0,0,0,0.00
8564,2022,8,aftonbladet.se,180x150,samsung,300,0,0,0,0.00
